In [1]:
import pynbody
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

%matplotlib inline

In [2]:
def loader(path):
    '''returns snapshot and halo'''
    snap=pynbody.load(path)
    snap.physical_units()
    h=snap.halos()
    return snap,h

def BH_finder(snap):
    '''returns subsnap of black holes'''
    return snap.star[pynbody.filt.BandPass('tform','-15 Gyr','0 Gyr')]

def analysis(halo, view):
    '''center snap on halo's center of mass (angmom gives errors if there aren't enough pixels)'''
    if view==1:
        pynbody.analysis.angmom.faceon(halo)
    elif view==2:
        pynbody.analysis.angmom.sideon(halo)
    else:
        pynbody.analysis.halo.center(halo)
        
def plot_BH_pos(h, BH, grpid, attribute, cmap='Greys', w=100, save=False, snapshot=4096, view=0):
    '''plots position of black holes. Send halo array(h), black hole array(BH), grpid, and attribute you want to plot 
    Optional: width/preferred scope(w) (default=100), colormap(default='Greys'), save(default=False), snapshot(default=4096)'''
    
    pynbody.plot.image(h[grpid].s,qty=attribute,cmap=cmap, width=w, log=False)
    plt.plot(np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[0],np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[1],'r+')
    plt.xlim(-w/2,w/2)
    plt.ylim(-w/2,w/2)
    plt.title('%s_h%s_%s_w%s'%(snapshot, grpid, attribute, w))
    
    if save==True:
        plt.savefig('plots/%s/h%s/h148_h%s_%s_w%s.png'%(snapshot, grpid, grpid, attribute, w))
    plt.show()

In [3]:
#returns snapshot and halos in physical units (takes a couple of minutes)
snap,h=loader('/data/scratch/jillian/h148/h148.cosmo50PLK.3072g3HbwK1BH.004096/h148.cosmo50PLK.3072g3HbwK1BH.004096')

In [4]:
#halo id for halo of interest
grpid=1
snap_id=4096

In [5]:
#angle of perspective (maybe?): 1 for face on, 2 for side on, anything else will leave it as is
view=2
analysis(h[grpid],view)

In [ ]:
#Optional: convert units
h[grpid].s['tform'].convert_units('Gyr')

In [6]:
#returns subsnap of black holes
BH=BH_finder(snap)

In [ ]:
#plot location of black holes
plot_BH_pos(h, BH, grpid, attribute='tform')

In [25]:
#create array for each halo to store masses at each snap
#h1_mass=pynbody.array.SimArray([], 'Msol')
#h1_mass['Gas', 'Star', 'Total']=h[grpid].g['mass'].sum(),h[grpid].s['mass'].sum(), h[grpid]['mass'].sum()

In [26]:
h1_mass_pd=pd.DataFrame(index=['4096'], columns=['Gas','Star','Total'])

In [27]:
h1_mass_pd['Gas']['4096']=h[grpid].g['mass'].sum()
h1_mass_pd['Star']['4096']=h[grpid].s['mass'].sum()
h1_mass_pd['Total']['4096']=h[grpid]['mass'].sum()

In [28]:
h1_mass_pd

,Gas,Star,Total
4096,1.67460004583e+11,1.88270265251e+11,2.10912325919e+12


In [39]:
x=pynbody.array.SimArray(h1_mass_pd, 'Msol')

In [36]:
np.array(h1_mass_pd).shape

(1, 3)

In [42]:
x.T

SimArray([[SimArray(167460004582.71896, 'Msol')],
       [SimArray(188270265250.5389, 'Msol')],
       [SimArray(2109123259192.6453, 'Msol')]], dtype=object, 'Msol')

In [44]:
x[0]

SimArray([SimArray(167460004582.71896, 'Msol'),
       SimArray(188270265250.5389, 'Msol'),
       SimArray(2109123259192.6453, 'Msol')], dtype=object, 'Msol')

In [46]:
h1_mass_pd['Gas']

4096    1.67460004583e+11
Name: Gas, dtype: object